In [33]:
from confpred import ConformalPredictor,SparseScore,SoftmaxScore
from confpred.classifier import load_model, evaluate
from confpred.datasets import load_dataset
#from confpred.utils import ROOT_DIR

from entmax.losses import SparsemaxLoss, Entmax15Loss
import torch
import numpy as np
import pandas as pd 
import os.path
import pickle
loss = 'entmax' #sparsemax, softmax or entmax15
model_type='vit'
transformation = 'logits'
dataset='ImageNet' #CIFAR100 or MNIST
seed='23'
alpha=0.1
#device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'
directory = '.'
fname = f'{directory}/data/predictions/{model_type}_{dataset}_test_{loss}_{transformation}_{seed}_proba.pickle'
print(fname)
if os.path.isfile(fname):
    print('Loading predictions.')
    path = f'{directory}/data/predictions/{model_type}_{dataset}_test_{loss}_{transformation}_{seed}_proba.pickle'
    with open(path, 'rb') as f:
        test_proba = pickle.load(f)
    path = f'{directory}/data/predictions/{dataset}_{seed}_test_true.pickle'
    with open(path, 'rb') as f:
        test_true_enc = pickle.load(f)
    path = f'{directory}/data/predictions/{model_type}_{dataset}_cal_{loss}_{transformation}_{seed}_proba.pickle'
    with open(path, 'rb') as f:
        cal_proba = pickle.load(f)
    path = f'{directory}/data/predictions/{dataset}_{seed}_cal_true.pickle'
    with open(path, 'rb') as f:
        cal_true_enc = pickle.load(f)
#Conformal Prediction
if loss == 'sparsemax':
    cp = ConformalPredictor(SparseScore(2))
elif loss == 'softmax':
    cp = ConformalPredictor(SoftmaxScore())
elif loss== 'entmax':
    cp = ConformalPredictor(SparseScore(1.5))

cp.calibrate(cal_true_enc, cal_proba, alpha)
#print('done cal')
#avg_set_size, coverage = cp.evaluate(test_true_enc, test_proba)

./data/predictions/vit_ImageNet_test_entmax_logits_23_proba.pickle
Loading predictions.


In [34]:
from entmax.activations import sparsemax, entmax15

qhat = cp.q_hat

In [35]:
pred_ent = entmax15((2/qhat)*torch.tensor(test_proba), dim=-1).numpy()
test_match = pred_ent>0
n_test = test_match.shape[0]
set_size_2 = test_match.sum(axis = 1).mean()
coverage_2 = test_match[test_true_enc.astype(bool)].sum()/n_test

In [36]:
print(set_size_2,set_size)

1.1446 1.6544


In [13]:
n_test = test_proba.shape[0]
#test_match = cp.predict(test_proba)
test_scores = cp.score.get_multiple_scores(test_proba)
#test_match = test_scores<= self.q_hat
#set_size = test_match.sum(axis = 1).mean()
#coverage = test_match[test_true.astype(bool)].sum()/n_test